In [ ]:
from google.colab import drive

# drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
!pip install -q pandas

In [ ]:
!pip install -q bitsandbytes transformers accelerate datasets peft

In [ ]:
!pip install -q torch==2.6.0

In [ ]:
!pip install -q python-multipart nest-asyncio python-dotenv ftfy lxml_html_clean huggingface_hub

In [ ]:
!pip install -q langchain-community chromadb langchain sentence-transformers

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Using CPU.")

GPU is available!
GPU Name: NVIDIA L4


In [ ]:
!nvidia-smi


Mon Jun 30 13:50:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   38C    P8             15W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("drive/MyDrive/google_colab/MT/secrets.env")
# authtoken = os.environ.get("HUGGINGFACE_HUB_TOKEN")
# print(authtoken)

True

In [ ]:
# # to download a model from hugging face

# from huggingface_hub import login
# login(os.getenv("HUGGINGFACE_TOKEN"))

# from huggingface_hub import snapshot_download

# local_dir = "drive/MyDrive/google_colab/MT/models/granite-3.3-8b-instruct"

# snapshot_download(
#     repo_id="ibm-granite/granite-3.3-8b-instruct",
#     local_dir=local_dir,
#     local_dir_use_symlinks=False,
#     token=True
# )

In [ ]:
questions_12 = [
      "What is the overall sentiment regarding Switzerland? (Very negative, Negative, Neutral, Positive, Very positive)",
      "Which aspect of the natural dimension is discussed? (Nature Dimension not addressed, Landscape/scenery, Geography, Weather/climate, Preserved nature, Nature activities, Other aspect of Nature dimension)",
      "What is the sentiment toward the natural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Which aspect of the functional dimension is discussed? (Functional Dimension not addressed, Education system, Science/innovation, Products, Economy, Infrastructure, Politics, Living/working conditions, Security, Other aspect of Functional dimension)",
      "What is the sentiment toward the functional dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Which aspect of the normative dimension is discussed? (Normative Dimension not addressed, Environmental protection, Freedom/human rights, Civil rights, International engagement, Ethical issues/scandals, Conflict avoidance, Tolerance/openness, Other aspect of Normative dimension)",
      "What is the sentiment toward the normative dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Which aspect of the cultural dimension is discussed? (Cultural Dimension not addressed, Sports, Food, Cultural offer, Personalities, Traditions, History, Cultural diversity, Other aspect of Culture dimension)",
      "What is the sentiment toward the cultural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Does the article contain disinformation? (No disinformation type, False connection, False context, Misleading content, Fabricated content, Manipulated content, Other disinformation type)",
      "What disinformation technique is used, if any? (No disinformation technique, Ad hominem attack, Emotional language, False dichotomies, Incoherence, Scapegoating, Other disinformation technique)",
      "What is the article about in short?"
    ]

columns_12 = [
        'overall_sentiment',
        'nature_dimension',
        'nature_sentiment',
        'functional_dimension',
        'functional_sentiment',
        'normative_dimension',
        'normative_sentiment',
        'cultural_dimension',
        'cultural_sentiment',
        'disinformation_type',
        'disinformation_technique',
        'summary'
    ]


questions_6 = [
        "What is the overall sentiment regarding Switzerland? (Very negative, Negative, Neutral, Positive, Very positive)",
        "What is the sentiment toward the natural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "What is the sentiment toward the functional dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "What is the sentiment toward the normative dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "What is the sentiment toward the cultural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "Does the article contain disinformation? (No disinformation type, False connection, False context, Misleading content, Fabricated content, Manipulated content, Other disinformation type)",
    ]

columns_6 = [
        'overall_sentiment',
        'nature_sentiment',
        'functional_sentiment',
        'normative_sentiment',
        'cultural_sentiment',
        'disinformation_type',
    ]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
def load_model(model_path):
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  tokenizer.pad_token = tokenizer.eos_token

  model = AutoModelForCausalLM.from_pretrained(
      model_path,
      device_map="auto",
      torch_dtype="auto",
      load_in_8bit=True
  )

  return model, tokenizer



In [ ]:
import re
import pandas as pd
def process_by_mistral_7b(model, tokenizer, input_file_path, questions, columns, output_file_path):

    data = pd.read_json(input_file_path, orient="records", lines=True)

    print(f"Data len: {len(data)}")

    data['article_source'] = ''
    for col in columns:
        data[col] = ''

    for index, row in data.iterrows():
        if not row['content_extracted'] or row['extracted_content'] == "" or len(row['content']) > len(row['extracted_content']):
            article = row['content']
            data.at[index, 'article_source'] = 'content'
        else:
            article = row['extracted_content']
            data.at[index, 'article_source'] = 'extracted_content'

        prompt = f"""<s>[INST]
        You are given an article about Switzerland. Read it carefully.

        ARTICLE:
        {article}

        TASK:
        Answer the following multiple-choice questions based only on the information in the article.

        Instructions:
        - Choose **only one** of the options provided for each question.
        - Write only the selected option.
        - **Do not** explain your answer.

        QUESTIONS:
        """
        for i, q in enumerate(questions, 1):
            prompt += f"{i}. {q}\n"

        prompt += "[/INST]"


        # Tokenize and send to model
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=512,
                pad_token_id=tokenizer.eos_token_id
                # do_sample=True,
                # temperature=0.7,
                # top_p=0.9,
                # top_k=50
            )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        response_text = decoded.replace(prompt, "").strip()
        # print("--------------------------------------------------")
        print("--------------------------------------------------")
        print(f"Index: {index}")

        # print(f"Response: {response_text}")
        # print("--------------------------------------------------")
        # print("--------------------------------------------------")

        answer_lines = re.findall(r"^\s*(\d+)\.\s*(.*)$", response_text, flags=re.MULTILINE)
        answers = {int(num): ans.strip() for num, ans in answer_lines}

        # Fill in DataFrame columns
        for i, col in enumerate(columns, 1):
            data.at[index, col] = answers.get(i, "")



    # Save updated DataFrame to CSV
    data.to_csv(output_file_path, index=False, encoding='utf-8')
    print("Processing complete. Saved to CSV.")


In [ ]:
# model, tokenizer = load_model("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-pre")

In [ ]:
# from datetime import datetime
# data_count = 6


# print("Mistral-Nemo-Instruct-2407")
# for i in range(1):
#     print(f"This is loop number {i + 1}")
#     timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#     print(timestamp)
#     process_by_mistral_7b(model=model,
#                   tokenizer=tokenizer,
#                   input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
#                   questions=questions_6,
#                   columns=columns_6,
#                   output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_mistral_nemo_{data_count}_{timestamp}.csv")
#     print(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
#     print("--------------------------------------------------")

In [ ]:
import json

with open("drive/MyDrive/google_colab/MT/uploads/training_data_658.json", "r", encoding="utf-8") as f:
    fine_tuning_data = json.load(f)


# print(len(fine_tuning_data))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType
from datasets import Dataset
import torch
import os

model_path = "drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-pre"
model, tokenizer = load_model(model_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:


# Prepare for k-bit training
model = prepare_model_for_kbit_training(model)

# Apply LoRA
lora_config = LoraConfig(
    r=16,  # Low-rank dimension (recommended 8-64)
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Mistral attention layer names
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

In [ ]:
# Prepare dataset
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

dataset = Dataset.from_dict({"text": fine_tuning_data}).map(tokenize_function, batched=True)

# Data collator for padding & masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

Map:   0%|          | 0/658 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./finetuned-mistral",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="./logs",
    save_strategy="epoch",
    report_to="none",
)

# Trainer setup
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
)

/tmp/ipython-input-23-3105373227.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Fine-tune
trainer.train()

In [ ]:
# Merge LoRA adapters into the base model for saving
model = model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:85: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# Save full model and tokenizer
model.save_pretrained("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-post-1")
tokenizer.save_pretrained("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-post-1")


('drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-post-1/tokenizer_config.json',
 'drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-post-1/special_tokens_map.json',
 'drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-post-1/chat_template.jinja',
 'drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-post-1/tokenizer.json')

In [ ]:
model_3, tokenizer_3 = load_model("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-post-1")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from datetime import datetime
data_count = 133


print("Mistral-Nemo-Instruct-2407-post-1")
for i in range(5):
    print(f"This is loop number {i + 1}")
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    print(timestamp)
    process_by_mistral_7b(model=model_3,
                  tokenizer=tokenizer_3,
                  input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
                  questions=questions_6,
                  columns=columns_6,
                  output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_mistral_nemo__post_1_{data_count}_{timestamp}.csv")
    print(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    print("--------------------------------------------------")

In [ ]:
# Load the model
model_path = "drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-pre"
model, tokenizer = load_model(model_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:


# question = "How much was the GDP of Switzerland in the last 5 years?"

# # Encode prompt
# inputs = tokenizer(
#     question,
#     return_tensors="pt",
#     padding=True
# ).to(model.device)

# # Generate output
# output = model.generate(
#     **inputs,
#     max_length=200,          # how long the answer can be
#     temperature=0.7,         # randomness of sampling
#     top_p=0.9,               # nucleus sampling
#     do_sample=True,          # enable sampling
#     pad_token_id=tokenizer.eos_token_id
# )

# # Decode tokens to text
# answer = tokenizer.decode(output[0], skip_special_tokens=True)

# # Print the response
# print(answer)

In [ ]:
documents_content = [
    "Switzerland is a mountainous country in Central Europe, known for its neutrality and banking sector.",
    "The Swiss Alps attract millions of tourists each year for skiing and hiking.",
    "Zurich is Switzerland's largest city and an important financial hub.",
    "The Swiss Confederation was founded in 1291.",
    "Switzerland’s nominal GDP is expected to continue its steady climb. According to the IMF’s World Economic Outlook (April 2025), Switzerland’s GDP at current prices is projected at USD 947.1 billion in 2025. With modest real growth of around 1.2–1.6% expected in 2026 and assuming stable prices and exchange rates, the nominal GDP value for 2026 is likely to reach approximately USD 960–970 billion. For context, the economy was estimated at USD 884.9 billion in 2023 and climbed to around USD 908.5 billion in 2025, according to World Economics . In summary, Switzerland nominal GDP in 2026 is expected to fall in the USD 960–970 billion range.",
    "Switzerland has four national languages: German, French, Italian, and Romansh.",
    "Swiss watches are renowned worldwide for their precision and craftsmanship.",
    "The country is famous for its high-quality chocolate production."
]

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document

# Create embeddings
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create ChromaDB vector store
vector_store = Chroma.from_texts(
    texts=documents_content,
    embedding=embedding_model,
    collection_name="swiss_knowledge"
)

# Retrieval function
def retrieve(query, top_k=2):
    results = vector_store.similarity_search(query, k=top_k)
    return [doc.page_content for doc in results]


def generate_answer(query):
    retrieved = retrieve(query)
    context = "\n".join(retrieved)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

query = "How much the GDP of Switzerland in 2026?"
print(generate_answer(query))


/tmp/ipython-input-25-1313687663.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Context:
Switzerland’s nominal GDP is expected to continue its steady climb. According to the IMF’s World Economic Outlook (April 2025), Switzerland’s GDP at current prices is projected at USD 947.1 billion in 2025. With modest real growth of around 1.2–1.6% expected in 2026 and assuming stable prices and exchange rates, the nominal GDP value for 2026 is likely to reach approximately USD 960–970 billion. For context, the economy was estimated at USD 884.9 billion in 2023 and climbed to around USD 908.5 billion in 2025, according to World Economics . In summary, Switzerland nominal GDP in 2026 is expected to fall in the USD 960–970 billion range.
Zurich is Switzerland's largest city and an important financial hub.

Question: How much the GDP of Switzerland in 2026?
Answer: The GDP of Switzerland in 2026 is expected to be around USD 960–970 billion.
